<a href="https://www.kaggle.com/code/averma111/data-centric-pss3e21?scriptVersionId=140709966" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

from category_encoders import OneHotEncoder, MEstimateEncoder, GLMMEncoder, OrdinalEncoder, CatBoostEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import  RandomForestRegressor
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve,mean_absolute_error
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, PowerTransformer

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
seed=42

/kaggle/input/playground-series-s3e21/sample_submission.csv


In [20]:
ROOT_PATH = '/kaggle/input/playground-series-s3e21'
submission_df = pd.read_csv(ROOT_PATH+'/sample_submission.csv')

In [21]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    return summ

In [22]:
submission_df.head()

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.00,9.545,9.265,8.110,8.43,7.15,0.180,...,4.95,1.730,1.800,4.800,3.15,10.665,10.465,16.645,5.75,10.37
1,1,9.10,13.533,40.90,8.770,9.265,6.015,10.07,7.15,1.107,...,20.05,9.530,7.695,4.550,6.95,2.040,5.200,5.725,2.95,2.23
2,2,8.21,3.710,5.42,8.770,9.265,4.550,10.07,7.15,0.020,...,4.58,3.025,3.960,4.935,4.95,4.725,6.075,6.750,3.50,3.17
3,3,8.39,8.700,8.10,9.500,9.200,5.200,8.67,6.67,0.280,...,8.45,2.070,1.730,6.300,4.70,3.500,6.200,8.670,2.90,7.37
4,4,8.07,8.050,8.65,7.960,9.265,3.290,10.07,7.15,0.360,...,2.02,1.730,0.760,4.800,4.97,3.950,2.800,8.400,3.50,3.90


In [23]:
summary('submission',submission_df)

submission shape: (3500, 37)


,dtypes,null,unique,min,median,max,mean,std
id,int64,0,3500,0.000,1749.500,3499.000,1749.500000,1010.507298
target,float64,0,462,1.300,8.280,65.930,8.473834,1.885712
O2_1,float64,0,329,0.000,7.890,46.950,8.216993,3.040743
O2_2,float64,0,258,0.000,8.300,65.950,9.291581,6.817822
O2_3,float64,0,235,4.900,9.500,16.900,9.633064,1.439039
O2_4,float64,0,241,2.300,7.980,21.800,8.065733,1.463881
O2_5,float64,0,317,0.200,5.800,59.400,5.671581,2.721399
O2_6,float64,0,238,0.000,9.430,40.190,9.461363,1.140365
O2_7,float64,0,303,0.000,6.430,15.900,6.547433,1.663266
NH4_1,float64,0,152,0.020,0.247,4.200,0.340525,0.284751


In [24]:
def removenan(df,column):
        df[column].fillna(df[column].mean(), inplace=True)
        return df
    

for cols in submission_df.columns:
    if submission_df[cols].isnull().sum()>0:
        submission_df=removenan(submission_df,cols)

In [25]:
X = submission_df.copy()
y = X.pop('target')
k = LeaveOneGroupOut()

In [26]:
def date_processing(x):
    x_copy = x.copy()
    return x_copy

dtpr = FunctionTransformer(date_processing)

In [27]:
def select_features(x):
    return x[['O2_1', 'O2_2', 'O2_3', 'O2_4',
                   'NH4_2', 'NH4_5', 'NH4_6',
                   'NO2_1', 'NO2_2', 'NO2_3', 'NO3_3',
                   'BOD5_1', 'BOD5_5']]
rmfea = FunctionTransformer(select_features)

In [40]:
def cross_val_score(model, cv = k, label = ''):
    
    X = submission_df.copy()
    y = X.pop('target')
    
    val_predictions = np.zeros((len(submission_df)))
    train_scores, val_scores = [], []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, X.id)):

        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        

        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]
        

        model.fit(X_train, y_train)
        

        train_preds = model.predict(X_train)
        val_preds = model.predict(X_val)
                  
        val_predictions[val_idx] += val_preds
        

        train_score = mean_absolute_error(y_train, train_preds)
        val_score = mean_absolute_error(y_val, val_preds)

        train_scores.append(train_score)
        val_scores.append(val_score)
    
    print(f' Train Score: {np.mean(train_scores):.5f} ± {np.std(train_scores):.5f} | Val Score: {np.mean(val_scores):.5f} ± {np.std(val_scores):.5f} | {label}')
    
    return val_scores, val_predictions

In [41]:
score_list, off_list = pd.DataFrame(), pd.DataFrame()

models = [
    (
    ('rf', RandomForestRegressor(random_state = seed,n_estimators=1000,
       max_depth=7,
       n_jobs=-1))
    )
]

In [ ]:
for (label, model) in models:
     score_list[label], off_list[label] = cross_val_score(
         make_pipeline(dtpr, rmfea, model),
         label = label,
     )

In [ ]:
plt.figure(figsize = (8, 4), dpi = 150)
sns.barplot(data = score_list.reindex((score_list).mean().sort_values().index, axis = 1), palette = 'rocket', orient = 'v')
plt.title('Score Comparison', size = 10)
plt.show()

In [ ]:
pred_train = rf.predict(df[features_select])

In [ ]:
plt.scatter(y_train, pred_train, color=default_color_3, alpha=.25)
plt.title('Predicted vs Actual - Training')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid()
plt.show()

In [ ]:
submission = df.copy()

# set columns to zero for features that were not selected
for f in features:
    if (f not in features_select):
        submission[f] = 0
        
submission.head()

In [ ]:
submission.to_csv('submission.csv', index = False)